---
draft: true
title: Finding pole positions
---

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from IPython.display import Math

In [ ]:
n = 1
K = sp.MatrixSymbol("K", n, n)
rho = sp.MatrixSymbol("rho", n, n)
I = sp.Identity(n)

In [ ]:
T = (I + sp.I * rho * K).inv() * K
T

In [ ]:
T.as_explicit()

In [ ]:
s, g1, m1, g2, m2 = sp.symbols("s g1 m1 g2 m2")
ma, mb = sp.symbols("ma mb")

In [ ]:
substitutions = {
    K[0, 0]: g1**2 / (m1**2 - s) + g2**2 / (m2**2 - s),
    rho[0, 0]: sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2)) / s,
}
Math(aslatex(substitutions))

In [ ]:
T_expr = T.as_explicit().subs(substitutions)[0].simplify()
T_expr

In [ ]:
numerator, denominator = sp.fraction(T_expr)
denominator

In [ ]:
parameters = {
    "m1": 1.8,
    "m2": 1.1,
    "g1": 0.5,
    "g2": 0.7,
    "ma": 0,
    "mb": 0,
}

In [ ]:
T_func = sp.lambdify(s, T_expr.subs(parameters))

In [ ]:
Denom_func = sp.lambdify(s, denominator.subs(parameters))

In [ ]:
x = np.linspace(0, 5, 500)
y = T_func(x - 1e-8j)
plt.plot(x, np.abs(y))
plt.show()

In [ ]:
X, Y = np.meshgrid(
    np.linspace(-1, 6, num=100),
    np.linspace(1, -2, num=100),
)
S = X + 1j * Y
Z = T_func(S)
mesh = plt.pcolormesh(X, Y, np.imag(Z), vmin=-1, vmax=1)
plt.colorbar(mesh)
plt.show()

In [ ]:
X, Y = np.meshgrid(
    np.linspace(0, 10, num=100),
    np.linspace(0, 10, num=100),
)
S = X + 1j * Y
Z = Denom_func(S)
mesh = plt.pcolormesh(X, Y, np.abs(Z))
plt.colorbar(mesh)
plt.show()

In [ ]:
sp.solve(sp.Eq(denominator.subs(parameters), 0), s)